In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import tqdm
import numpy as np

In [20]:
transform = transforms.Compose(
    [transforms.Resize(size = (224,224)), 
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [21]:
train = '1Amostragem/train'
test = '1Amostragem/test'

In [22]:
train_dataset = datasets.ImageFolder(train,transform = transform)
test_dataset = datasets.ImageFolder(test,transform = transform)

In [23]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 50
    Root location: 1Amostragem/train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [24]:
train_dataset.classes

['normal', 'pneumonia']

In [25]:
test_dataset

Dataset ImageFolder
    Number of datapoints: 50
    Root location: 1Amostragem/test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [26]:
test_dataset.classes

['normal', 'pneumonia']

In [27]:
trainloader = DataLoader(train_dataset,batch_size = 64,shuffle = True)
testloader = DataLoader(test_dataset,batch_size = 64,shuffle = True)

In [28]:
images, labels = next(iter(trainloader))
print(images.shape)
print(labels.shape)

torch.Size([50, 3, 224, 224])
torch.Size([50])


In [29]:
from torch.autograd import Variable
for i, (images,labels) in enumerate(trainloader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())

In [30]:
class Model(nn.Module):
    def __init__(self, num_classes=2):
        super(Model, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=96, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(96)
        self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(256)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(384)
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(384)
        self.conv5 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.linear1 = nn.Linear(256 * 14 * 14, 1000)
        self.bn6 = nn.BatchNorm1d(1000)
        self.linear2 = nn.Linear(1000, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.relu(self.bn5(self.conv5(x)))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.relu(self.bn6(self.linear1(x)))
        x = self.linear2(x)
        return x

# Use CUDA if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [35]:
model = Model(num_classes=2)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 20
train_losses = []
test_losses = []

for epoch in range(num_epochs):
    model.train()  
    running_loss = 0
    
    for images, labels in trainloader:
        
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    avg_train_loss = running_loss / len(trainloader)
    train_losses.append(avg_train_loss)
    
    print(f"Epoca {epoch + 1}/{num_epochs} - Perda no treinamento: {avg_train_loss:.4f}")
    
    # Teste
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    avg_test_loss = test_loss / len(testloader)
    test_losses.append(avg_test_loss)
    accuracy = correct / total * 100
    
    print(f"Epoca {epoch + 1}/{num_epochs} - Perda no teste: {avg_test_loss:.4f}, Metrica de avaliacao (Accuracy): {accuracy:.2f}%")

Epoca 1/20 - Perda no treinamento: 0.6766
Epoca 1/20 - Perda no teste: 1.1064, Metrica de avaliacao (Accuracy): 50.00%
Epoca 2/20 - Perda no treinamento: 0.1084
Epoca 2/20 - Perda no teste: 1.3317, Metrica de avaliacao (Accuracy): 44.00%
Epoca 3/20 - Perda no treinamento: 1.3217
Epoca 3/20 - Perda no teste: 2.2411, Metrica de avaliacao (Accuracy): 52.00%
Epoca 4/20 - Perda no treinamento: 0.0731
Epoca 4/20 - Perda no teste: 2.6876, Metrica de avaliacao (Accuracy): 52.00%
Epoca 5/20 - Perda no treinamento: 0.0676
Epoca 5/20 - Perda no teste: 3.1202, Metrica de avaliacao (Accuracy): 52.00%
Epoca 6/20 - Perda no treinamento: 0.0666
Epoca 6/20 - Perda no teste: 2.6031, Metrica de avaliacao (Accuracy): 54.00%
Epoca 7/20 - Perda no treinamento: 0.0544
Epoca 7/20 - Perda no teste: 1.7585, Metrica de avaliacao (Accuracy): 60.00%
Epoca 8/20 - Perda no treinamento: 0.0471
Epoca 8/20 - Perda no teste: 1.4023, Metrica de avaliacao (Accuracy): 66.00%
Epoca 9/20 - Perda no treinamento: 0.0408
Epoca 